In [1]:
import os
import datetime
import pathlib
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# 取得巴哈動畫瘋官網 HTML
req_anigamer = requests.get('https://ani.gamer.com.tw/')
soup = BeautifulSoup(req_anigamer.text, 'html.parser')
thisSeasonAnime = soup.find_all(attrs={"data-look": "本季新番"})

## 擷取資訊項目
numAni = [x for x in range(len(thisSeasonAnime))]
title = [str(thisSeasonAnime[i].find('p').string) for i in numAni]
vol = [thisSeasonAnime[i].find(
    class_="newanime-vol").string.replace("第 ", "").replace(" 集", "") for i in numAni]
update = [thisSeasonAnime[i].find(
    class_="newanime-date").string.replace(" 更新", "") for i in numAni]
untrt_number = [thisSeasonAnime[i].find(
    class_="newanime-count") for i in numAni]
number = []
for str_temp in untrt_number:
    posi_end = str(str_temp).find("</span>")
    posi_stat = str(str_temp).rfind(">", 0, posi_end)
    num_temp = int(str(str_temp)[posi_stat+1:posi_end].replace(",", ""))
    number.append(num_temp)

## 將擷取資料彙整於表格
AnimeInfo = pd.DataFrame(
    {'title': title,
     'vol': vol,
     'update': update,
     'number': number
     })
### 加入彙整時間戳記
AnimeInfo['date'] = datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
AnimeInfo.head()

,title,vol,update,number,date
0,戀愛幕末男友外傳-危機,2,04/12,2149,2019/04/12 18:28:46
1,Fight League 交鋒聯盟,9,04/11,57250,2019/04/12 18:28:46
2,一拳超人 (第二季),13,04/10,356899,2019/04/12 18:28:46
3,強襲魔女 501 部隊出動！,1,04/10,14948,2019/04/12 18:28:46
4,鑽石王牌 act2,2,04/09,58901,2019/04/12 18:28:46


In [3]:
# 存檔
tsp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
if not os.path.exists(pathlib.PurePath('DailyData/')):
    os.makedirs(pathlib.PurePath('DailyData/'))
AnimeInfo.to_csv(pathlib.PurePath('DailyData/AniGamerInfo '+tsp+'.csv'))